## dataset handler (folder-based)

In [1]:
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import config as cfg

from data_loader.dataset import dset_imageFolder

In [2]:
# Load Dataset from folders
train_dataset, valid_dataset, test_dataset = dset_imageFolder()

train_loader = DataLoader(dataset=train_dataset, batch_size=cfg.batch_size, shuffle=cfg.shuffle, num_workers=cfg.num_workers)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=cfg.batch_size, shuffle=cfg.shuffle, num_workers=cfg.num_workers)
test_loader = DataLoader(dataset=test_dataset, batch_size=cfg.batch_size, shuffle=cfg.shuffle, num_workers=cfg.num_workers)

In [4]:
# create train and valid dataset using train_dataset and random split
# len_train = round(len(train_dataset) * 0.7)
# len_valid = len(train_dataset) - len_train

# train_dataset, valid_dataset = random_split(train_dataset, 
#                                             lengths=[len_train, len_valid], 
#                                             generator=torch.Generator().manual_seed(1))

In [6]:
# demo for handler and loader
samples = next(iter(train_loader))
print(samples[0].shape)

torch.Size([2, 3, 128, 128])


In [7]:
# create deep learning models : LeNet-5

import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    
    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.convolutions = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features=75000, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes)
        )
    
    def forward(self, x):
        x = self.convolutions(x)
        x = torch.flatten(x,1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return probs

In [8]:
model = LeNet5(n_classes=len(train_dataset.dataset.classes))
print(model)
model(samples[0])

LeNet5(
  (convolutions): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=75000, out_features=84, bias=True)
    (1): Tanh()
    (2): Linear(in_features=84, out_features=3, bias=True)
  )
)


tensor([[0.3202, 0.3324, 0.3473],
        [0.3163, 0.3398, 0.3439]], grad_fn=<SoftmaxBackward0>)

In [9]:
# device, cuda or cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# loss function
loss_fn = torch.nn.CrossEntropyLoss()

# optimization algorithm
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train(True)

LeNet5(
  (convolutions): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=75000, out_features=84, bias=True)
    (1): Tanh()
    (2): Linear(in_features=84, out_features=3, bias=True)
  )
)

In [11]:
# implement training process
# use tqdm for progress bar
from tqdm.notebook import tqdm
# epoch loop
epochs = 5

# training epoch
for epoch in tqdm(range(0, epochs)):
    
    # training batch 
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):    

        optimizer.zero_grad() # remove gradients from previous loop
        
        outputs = model(inputs) # classify inputs
        
        loss = loss_fn(outputs, targets) # calculate loss between outputs and targets
        
        loss.backward() # implement backpropagation
        
        optimizer.step() # update weights/parameters
        
        break
    
    # run validate model with validation data
    # model.train(False)
    model.eval()
    for batch_idx, (inputs, targets) in enumerate(valid_loader):
        break

  0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
print(outputs[0])

tensor([0.4681, 0.3260, 0.2060], grad_fn=<SelectBackward0>)


In [ ]:
# implement data/, loss/, models/, optimizer/, scheduler/, tools/ or utilities/

# implement train.py

# implement test.py
